In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import OrdinalEncoder, StandardScaler,LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import math
from catboost import CatBoostRegressor
from sklearn import ensemble
import ylearn
from ylearn.causal_discovery import CausalDiscovery

In [2]:
import pandas as pd
np.random.seed(2022)

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
final_train_cols = [x for x in train.columns if x not in set(['treatment','outcome'])]
final_cols = final_train_cols

In [5]:
for col in train.columns:
    if train[col].dtype == 'object':
        label_encoder = LabelEncoder().fit(train[col])
        train[col] = label_encoder.transform(train[col])
        test[col] = label_encoder.transform(test[col])

In [6]:
train_na_dic = {}
test_na_dic = {}
for col in train.columns:
    train_na_dic[col] = train[col].isna().sum()/train.shape[0]
    
for col in test.columns:
    test_na_dic[col] = test[col].isna().sum()/test.shape[0]

In [7]:
na_fill_need_col = [x[0] for x in train_na_dic.items() if x[1]>0]

from xgboost.sklearn import XGBClassifier,XGBRegressor
training_col = [x for x in train.columns if x not in ['treatment','outcome']]
train_cols = ['V_8', 'V_37', 'V_25', 'V_7', 'V_6', 'V_30', 'V_14', 'V_31', 'V_12', 'V_39', 'V_32', 'V_15', 'V_16', 'V_2','V_24']
for col in na_fill_need_col:
    train_cols = [x for x in training_col if x != col and 'predict' not in x]
    train_na_df = train[~train[col].isna()]
    xgb = XGBRegressor(learning_rate = 0.3,max_depth = 5,min_child_weight = 5,
                        colsample_bytree = 0.7,
                        subsample = 0.91,
                        n_estimators = 100)
    cab = CatBoostRegressor()
    xgb.fit(train_na_df[train_cols],train_na_df[col])
    cab.fit(train_na_df[train_cols],train_na_df[col])
    train[col + '_predict'] = (xgb.predict(train[train_cols]) + cab.predict(train[train_cols]))/2
    test[col + '_predict'] = (xgb.predict(test[train_cols]) + cab.predict(test[train_cols]))/2

Learning rate set to 0.07218
0:	learn: 5.2495412	total: 178ms	remaining: 2m 57s
1:	learn: 5.0920929	total: 189ms	remaining: 1m 34s
2:	learn: 4.9458465	total: 200ms	remaining: 1m 6s
3:	learn: 4.8101615	total: 212ms	remaining: 52.9s
4:	learn: 4.6917501	total: 223ms	remaining: 44.4s
5:	learn: 4.5781612	total: 233ms	remaining: 38.5s
6:	learn: 4.4712837	total: 240ms	remaining: 34.1s
7:	learn: 4.3784115	total: 249ms	remaining: 30.8s
8:	learn: 4.2960921	total: 258ms	remaining: 28.4s
9:	learn: 4.2098359	total: 267ms	remaining: 26.4s
10:	learn: 4.1336672	total: 274ms	remaining: 24.6s
11:	learn: 4.0639756	total: 282ms	remaining: 23.2s
12:	learn: 3.9984103	total: 289ms	remaining: 21.9s
13:	learn: 3.9363037	total: 298ms	remaining: 21s
14:	learn: 3.8814958	total: 305ms	remaining: 20s
15:	learn: 3.8249304	total: 313ms	remaining: 19.2s
16:	learn: 3.7752781	total: 319ms	remaining: 18.5s
17:	learn: 3.7297354	total: 327ms	remaining: 17.8s
18:	learn: 3.6882982	total: 334ms	remaining: 17.3s
19:	learn: 3.6

170:	learn: 1.4469050	total: 1.49s	remaining: 7.23s
171:	learn: 1.4395843	total: 1.5s	remaining: 7.22s
172:	learn: 1.4315663	total: 1.51s	remaining: 7.2s
173:	learn: 1.4252058	total: 1.51s	remaining: 7.18s
174:	learn: 1.4168065	total: 1.52s	remaining: 7.17s
175:	learn: 1.4083669	total: 1.53s	remaining: 7.16s
176:	learn: 1.4007696	total: 1.53s	remaining: 7.14s
177:	learn: 1.3948056	total: 1.54s	remaining: 7.12s
178:	learn: 1.3880277	total: 1.55s	remaining: 7.1s
179:	learn: 1.3800764	total: 1.55s	remaining: 7.08s
180:	learn: 1.3735676	total: 1.56s	remaining: 7.07s
181:	learn: 1.3676074	total: 1.57s	remaining: 7.05s
182:	learn: 1.3603712	total: 1.57s	remaining: 7.03s
183:	learn: 1.3534887	total: 1.58s	remaining: 7.02s
184:	learn: 1.3469596	total: 1.59s	remaining: 7s
185:	learn: 1.3387668	total: 1.6s	remaining: 6.99s
186:	learn: 1.3320749	total: 1.6s	remaining: 6.97s
187:	learn: 1.3258245	total: 1.61s	remaining: 6.96s
188:	learn: 1.3178494	total: 1.64s	remaining: 7.02s
189:	learn: 1.311648

356:	learn: 0.6264322	total: 2.76s	remaining: 4.97s
357:	learn: 0.6240206	total: 2.77s	remaining: 4.96s
358:	learn: 0.6216109	total: 2.77s	remaining: 4.95s
359:	learn: 0.6189246	total: 2.78s	remaining: 4.94s
360:	learn: 0.6171564	total: 2.78s	remaining: 4.93s
361:	learn: 0.6146566	total: 2.79s	remaining: 4.92s
362:	learn: 0.6123328	total: 2.79s	remaining: 4.9s
363:	learn: 0.6101184	total: 2.8s	remaining: 4.89s
364:	learn: 0.6076647	total: 2.81s	remaining: 4.88s
365:	learn: 0.6055192	total: 2.81s	remaining: 4.87s
366:	learn: 0.6032109	total: 2.81s	remaining: 4.86s
367:	learn: 0.6009329	total: 2.82s	remaining: 4.84s
368:	learn: 0.5983002	total: 2.83s	remaining: 4.83s
369:	learn: 0.5961498	total: 2.83s	remaining: 4.82s
370:	learn: 0.5943418	total: 2.84s	remaining: 4.81s
371:	learn: 0.5920276	total: 2.84s	remaining: 4.8s
372:	learn: 0.5902730	total: 2.85s	remaining: 4.79s
373:	learn: 0.5882939	total: 2.85s	remaining: 4.77s
374:	learn: 0.5860543	total: 2.86s	remaining: 4.76s
375:	learn: 0.5

528:	learn: 0.3692339	total: 3.67s	remaining: 3.27s
529:	learn: 0.3683954	total: 3.67s	remaining: 3.26s
530:	learn: 0.3677253	total: 3.68s	remaining: 3.25s
531:	learn: 0.3667004	total: 3.69s	remaining: 3.24s
532:	learn: 0.3658212	total: 3.69s	remaining: 3.23s
533:	learn: 0.3649190	total: 3.69s	remaining: 3.22s
534:	learn: 0.3641741	total: 3.7s	remaining: 3.22s
535:	learn: 0.3636778	total: 3.71s	remaining: 3.21s
536:	learn: 0.3628470	total: 3.71s	remaining: 3.2s
537:	learn: 0.3621580	total: 3.71s	remaining: 3.19s
538:	learn: 0.3614919	total: 3.72s	remaining: 3.18s
539:	learn: 0.3606444	total: 3.73s	remaining: 3.17s
540:	learn: 0.3599005	total: 3.73s	remaining: 3.16s
541:	learn: 0.3590634	total: 3.73s	remaining: 3.16s
542:	learn: 0.3582425	total: 3.74s	remaining: 3.15s
543:	learn: 0.3576129	total: 3.74s	remaining: 3.14s
544:	learn: 0.3567188	total: 3.75s	remaining: 3.13s
545:	learn: 0.3557581	total: 3.75s	remaining: 3.12s
546:	learn: 0.3552030	total: 3.76s	remaining: 3.11s
547:	learn: 0.

688:	learn: 0.2793003	total: 4.59s	remaining: 2.07s
689:	learn: 0.2788607	total: 4.59s	remaining: 2.06s
690:	learn: 0.2785242	total: 4.6s	remaining: 2.06s
691:	learn: 0.2782783	total: 4.61s	remaining: 2.05s
692:	learn: 0.2779095	total: 4.61s	remaining: 2.04s
693:	learn: 0.2775625	total: 4.62s	remaining: 2.04s
694:	learn: 0.2772689	total: 4.62s	remaining: 2.03s
695:	learn: 0.2770386	total: 4.63s	remaining: 2.02s
696:	learn: 0.2766133	total: 4.63s	remaining: 2.01s
697:	learn: 0.2762512	total: 4.64s	remaining: 2.01s
698:	learn: 0.2759085	total: 4.64s	remaining: 2s
699:	learn: 0.2753860	total: 4.65s	remaining: 1.99s
700:	learn: 0.2750930	total: 4.65s	remaining: 1.99s
701:	learn: 0.2746932	total: 4.66s	remaining: 1.98s
702:	learn: 0.2743683	total: 4.66s	remaining: 1.97s
703:	learn: 0.2741220	total: 4.67s	remaining: 1.96s
704:	learn: 0.2736781	total: 4.67s	remaining: 1.96s
705:	learn: 0.2733713	total: 4.68s	remaining: 1.95s
706:	learn: 0.2730994	total: 4.68s	remaining: 1.94s
707:	learn: 0.27

...

347:	learn: 178.9081309	total: 2.01s	remaining: 3.76s
348:	learn: 178.8479393	total: 2.02s	remaining: 3.76s
349:	learn: 178.8061243	total: 2.02s	remaining: 3.75s
350:	learn: 178.7540955	total: 2.03s	remaining: 3.75s
351:	learn: 178.7127875	total: 2.03s	remaining: 3.74s
352:	learn: 178.6545825	total: 2.04s	remaining: 3.73s
353:	learn: 178.5810509	total: 2.04s	remaining: 3.73s
354:	learn: 178.5471857	total: 2.05s	remaining: 3.72s
355:	learn: 178.5131094	total: 2.05s	remaining: 3.72s
356:	learn: 178.4498931	total: 2.06s	remaining: 3.71s
357:	learn: 178.4312966	total: 2.06s	remaining: 3.7s
358:	learn: 178.3855731	total: 2.07s	remaining: 3.7s
359:	learn: 178.3568717	total: 2.08s	remaining: 3.69s
360:	learn: 178.3357108	total: 2.08s	remaining: 3.68s
361:	learn: 178.2292946	total: 2.09s	remaining: 3.68s
362:	learn: 178.1899186	total: 2.09s	remaining: 3.67s
363:	learn: 178.1355831	total: 2.1s	remaining: 3.67s
364:	learn: 178.0566083	total: 2.1s	remaining: 3.66s
365:	learn: 178.0379637	total: 2

502:	learn: 171.6083423	total: 2.92s	remaining: 2.88s
503:	learn: 171.5756132	total: 2.93s	remaining: 2.88s
504:	learn: 171.5450312	total: 2.93s	remaining: 2.87s
505:	learn: 171.5438991	total: 2.94s	remaining: 2.87s
506:	learn: 171.5078385	total: 2.94s	remaining: 2.86s
507:	learn: 171.4844693	total: 2.95s	remaining: 2.86s
508:	learn: 171.4702450	total: 2.96s	remaining: 2.85s
509:	learn: 171.4110559	total: 2.96s	remaining: 2.84s
510:	learn: 171.3988028	total: 2.97s	remaining: 2.84s
511:	learn: 171.3650601	total: 2.97s	remaining: 2.83s
512:	learn: 171.3453994	total: 2.98s	remaining: 2.83s
513:	learn: 171.3141371	total: 2.98s	remaining: 2.82s
514:	learn: 171.2672884	total: 2.99s	remaining: 2.81s
515:	learn: 171.1977725	total: 2.99s	remaining: 2.81s
516:	learn: 171.1926331	total: 3s	remaining: 2.8s
517:	learn: 171.1715140	total: 3s	remaining: 2.8s
518:	learn: 171.1259292	total: 3.01s	remaining: 2.79s
519:	learn: 171.0895586	total: 3.02s	remaining: 2.79s
520:	learn: 171.0418192	total: 3.02s

661:	learn: 165.9453255	total: 3.83s	remaining: 1.96s
662:	learn: 165.9044846	total: 3.83s	remaining: 1.95s
663:	learn: 165.8378960	total: 3.84s	remaining: 1.94s
664:	learn: 165.8125775	total: 3.85s	remaining: 1.94s
665:	learn: 165.7797930	total: 3.85s	remaining: 1.93s
666:	learn: 165.7467967	total: 3.86s	remaining: 1.93s
667:	learn: 165.7316275	total: 3.86s	remaining: 1.92s
668:	learn: 165.7103206	total: 3.87s	remaining: 1.91s
669:	learn: 165.6790192	total: 3.87s	remaining: 1.91s
670:	learn: 165.6416265	total: 3.88s	remaining: 1.9s
671:	learn: 165.6047849	total: 3.88s	remaining: 1.9s
672:	learn: 165.5682900	total: 3.89s	remaining: 1.89s
673:	learn: 165.5260418	total: 3.9s	remaining: 1.89s
674:	learn: 165.4885315	total: 3.9s	remaining: 1.88s
675:	learn: 165.4482771	total: 3.91s	remaining: 1.87s
676:	learn: 165.4124281	total: 3.92s	remaining: 1.87s
677:	learn: 165.3826556	total: 3.93s	remaining: 1.86s
678:	learn: 165.3554215	total: 3.93s	remaining: 1.86s
679:	learn: 165.3296182	total: 3

828:	learn: 160.6465760	total: 4.92s	remaining: 1.02s
829:	learn: 160.5969507	total: 4.94s	remaining: 1.01s
830:	learn: 160.5203342	total: 4.95s	remaining: 1s
831:	learn: 160.4798144	total: 4.96s	remaining: 1s
832:	learn: 160.4546252	total: 4.96s	remaining: 995ms
833:	learn: 160.4229173	total: 4.97s	remaining: 990ms
834:	learn: 160.3920246	total: 4.98s	remaining: 984ms
835:	learn: 160.3691830	total: 4.99s	remaining: 979ms
836:	learn: 160.3067446	total: 5s	remaining: 973ms
837:	learn: 160.2762471	total: 5.01s	remaining: 968ms
838:	learn: 160.2386669	total: 5.01s	remaining: 962ms
839:	learn: 160.2016578	total: 5.03s	remaining: 957ms
840:	learn: 160.1830680	total: 5.03s	remaining: 952ms
841:	learn: 160.1520886	total: 5.04s	remaining: 946ms
842:	learn: 160.1271374	total: 5.05s	remaining: 940ms
843:	learn: 160.0863671	total: 5.06s	remaining: 935ms
844:	learn: 160.0480396	total: 5.07s	remaining: 929ms
845:	learn: 160.0214854	total: 5.07s	remaining: 924ms
846:	learn: 159.9903928	total: 5.08s	

987:	learn: 155.7005034	total: 6.22s	remaining: 75.5ms
988:	learn: 155.6705213	total: 6.23s	remaining: 69.3ms
989:	learn: 155.6584730	total: 6.26s	remaining: 63.2ms
990:	learn: 155.6360663	total: 6.26s	remaining: 56.9ms
991:	learn: 155.6016788	total: 6.27s	remaining: 50.6ms
992:	learn: 155.5840764	total: 6.28s	remaining: 44.3ms
993:	learn: 155.5560321	total: 6.29s	remaining: 38ms
994:	learn: 155.5200481	total: 6.29s	remaining: 31.6ms
995:	learn: 155.4691242	total: 6.3s	remaining: 25.3ms
996:	learn: 155.4437142	total: 6.31s	remaining: 19ms
997:	learn: 155.4121819	total: 6.32s	remaining: 12.7ms
998:	learn: 155.3760597	total: 6.33s	remaining: 6.33ms
999:	learn: 155.3596760	total: 6.33s	remaining: 0us


In [10]:
# f2 = pd.DataFrame()
# f2['cols'] = train_cols
# f2['f'] = xgb.feature_importances_

In [11]:
#print(f2.sort_values(by = 'f',ascending=False)['cols'][:15].tolist())

In [12]:
#f2.sort_values(by = 'f',ascending=False)

In [13]:
def fillna(x1,x2):
    if math.isnan(x1):
        return x2
    else:
        return x1
    
for col in na_fill_need_col:
    train[col] = train.apply(lambda x:fillna(x[col],x[col+'_predict']),axis=1) 
    test[col] = test.apply(lambda x:fillna(x[col],x[col+'_predict']),axis=1) 

In [14]:
final_cols = ['V_2','V_9','V_15', 'V_28','V_8','V_31','V_36','V_12']#'V_20', 'V_27', 'V_28', 'V_32','V_13' 'V_33',
#['V_20', 'V_27', 'V_28', 'V_32']
#['V_2', 'V_3','V_6', 'V_4','V_15','V_23', 'V_17', 'V_28', 'V_30', 'V_34']#, 'V_39' #19 # 'V_6', 23

In [15]:
# ('V_3', 'outcome', 0.6931720873001793),
#  ('V_8', 'outcome', 0.5552655128197247),
#  ('V_11', 'outcome', 0.3421277398513122),
#  ('V_13', 'outcome', 0.45759926494490255),
#  ('V_15', 'outcome', 0.2522125971761858),
#  ('V_18', 'outcome', 0.10708746501889874),
#  ('V_31', 'outcome', 0.1400190199779578),
#  ('V_32', 'outcome', 0.14721097219302964),
#  ('V_33', 'outcome', 0.4801109380678446),
#  ('V_39', 'outcome', 0.312648680580717)]

In [16]:
from sklearn.preprocessing import RobustScaler,StandardScaler
scaler_cols = ['V_9','V_28','V_31','V_36']

t = StandardScaler().fit(train[scaler_cols])
training = t.transform(train[scaler_cols])
testing = t.transform(test[scaler_cols])

In [17]:
train[scaler_cols] = training
test[scaler_cols] = testing

In [18]:
# train_new = train[train['treatment'] == 2].sample(15000).append(train[train['treatment'] != 2])
# outcome_train = train_new['outcome']
# train_new = train_new[final_cols + ['treatment']]

In [19]:
xgb = XGBRegressor(learning_rate = 0.3,max_depth = 5,min_child_weight = 5,
colsample_bytree = 0.7,
subsample = 0.91,
n_estimators = 100)

In [20]:
#训练
#V_11
#final_cols = ['V_2', 'V_5', 'V_6', 'V_8', 'V_10', 'V_12', 'V_16', 'V_17', 'V_20', 'V_21', 'V_24', 'V_26', 'V_27', 'V_28', 'V_29', 'V_32', 'V_35', 'V_36', 'V_37', 'V_38', 'V_39']
train_treat_ori = train['treatment']
outcome_train = train['outcome']
train = train[final_cols + ['treatment']]

cab = CatBoostRegressor()
xgb.fit(train,outcome_train)
cab.fit(train,outcome_train)

train['treatment'] = 1
train_treatment1_xgb = xgb.predict(train)
train_treatment1_cab = cab.predict(train)
train_ensemble_treat1 = (train_treatment1_xgb + train_treatment1_cab)/2

train['treatment'] = 2
train_treatment2_xgb = xgb.predict(train)
train_treatment2_cab = cab.predict(train)
train_ensemble_treat2 = (train_treatment2_xgb + train_treatment2_cab)/2

train['treatment'] = 0
train_treatment0_xgb = xgb.predict(train)
train_treatment0_cab = cab.predict(train)
train_ensemble_treat0 = (train_treatment1_xgb + train_treatment0_cab)/2
train['t1_pred'] = train_ensemble_treat1
train['t2_pred'] = train_ensemble_treat2
train['t0_pred'] = train_ensemble_treat0
train['t_ori'] = train_treat_ori
train['outcome'] = outcome_train

test = test[final_cols]
test['treatment'] = 1
test_treatment1_xgb = xgb.predict(test)
test_treatment1_cab = cab.predict(test)
test_ensemble_treat1 = (test_treatment1_xgb + test_treatment1_cab)/2

test['treatment'] = 2
test_treatment2_xgb = xgb.predict(test)
test_treatment2_cab = cab.predict(test)
test_ensemble_treat2 = (test_treatment2_xgb + test_treatment2_cab)/2

test['treatment'] = 0
test_treatment0_xgb = xgb.predict(test)
test_treatment0_cab = cab.predict(test)
test_ensemble_treat0 = (test_treatment0_xgb + test_treatment0_cab)/2

test['t1_pred'] = test_ensemble_treat1
test['t2_pred'] = test_ensemble_treat2
test['t0_pred'] = test_ensemble_treat0

Learning rate set to 0.072183
0:	learn: 17.1571750	total: 7.11ms	remaining: 7.1s
1:	learn: 16.7263139	total: 12.8ms	remaining: 6.38s
2:	learn: 16.3358333	total: 19.6ms	remaining: 6.5s
3:	learn: 15.9886576	total: 26.5ms	remaining: 6.59s
4:	learn: 15.6809960	total: 32.4ms	remaining: 6.44s
5:	learn: 15.4107647	total: 39.1ms	remaining: 6.48s
6:	learn: 15.1494297	total: 46ms	remaining: 6.53s
7:	learn: 14.9197062	total: 51.4ms	remaining: 6.37s
8:	learn: 14.7250109	total: 57.5ms	remaining: 6.34s
9:	learn: 14.5403540	total: 65.8ms	remaining: 6.51s
10:	learn: 14.3765823	total: 71.1ms	remaining: 6.39s
11:	learn: 14.2275363	total: 77.2ms	remaining: 6.36s
12:	learn: 14.1051928	total: 83ms	remaining: 6.3s
13:	learn: 13.9776296	total: 88.3ms	remaining: 6.22s
14:	learn: 13.8705938	total: 93.8ms	remaining: 6.16s
15:	learn: 13.7685689	total: 100ms	remaining: 6.15s
16:	learn: 13.6894330	total: 105ms	remaining: 6.09s
17:	learn: 13.6141895	total: 112ms	remaining: 6.09s
18:	learn: 13.5341480	total: 118ms	r

168:	learn: 11.7868739	total: 1.06s	remaining: 5.22s
169:	learn: 11.7823844	total: 1.07s	remaining: 5.22s
170:	learn: 11.7798427	total: 1.09s	remaining: 5.3s
171:	learn: 11.7781379	total: 1.1s	remaining: 5.31s
172:	learn: 11.7762598	total: 1.11s	remaining: 5.3s
173:	learn: 11.7750787	total: 1.13s	remaining: 5.36s
174:	learn: 11.7733976	total: 1.14s	remaining: 5.35s
175:	learn: 11.7709847	total: 1.14s	remaining: 5.34s
176:	learn: 11.7694144	total: 1.15s	remaining: 5.34s
177:	learn: 11.7682540	total: 1.16s	remaining: 5.34s
178:	learn: 11.7676260	total: 1.16s	remaining: 5.34s
179:	learn: 11.7656023	total: 1.17s	remaining: 5.33s
180:	learn: 11.7639709	total: 1.18s	remaining: 5.33s
181:	learn: 11.7621433	total: 1.19s	remaining: 5.33s
182:	learn: 11.7616183	total: 1.19s	remaining: 5.33s
183:	learn: 11.7596038	total: 1.2s	remaining: 5.33s
184:	learn: 11.7583236	total: 1.21s	remaining: 5.32s
185:	learn: 11.7559730	total: 1.21s	remaining: 5.31s
186:	learn: 11.7541694	total: 1.22s	remaining: 5.3

334:	learn: 11.5804110	total: 2.14s	remaining: 4.25s
335:	learn: 11.5794028	total: 2.15s	remaining: 4.25s
336:	learn: 11.5788662	total: 2.16s	remaining: 4.24s
337:	learn: 11.5779883	total: 2.16s	remaining: 4.24s
338:	learn: 11.5774645	total: 2.17s	remaining: 4.23s
339:	learn: 11.5774039	total: 2.18s	remaining: 4.22s
340:	learn: 11.5764584	total: 2.18s	remaining: 4.21s
341:	learn: 11.5750353	total: 2.19s	remaining: 4.21s
342:	learn: 11.5743343	total: 2.19s	remaining: 4.2s
343:	learn: 11.5736947	total: 2.2s	remaining: 4.19s
344:	learn: 11.5721070	total: 2.2s	remaining: 4.18s
345:	learn: 11.5720222	total: 2.21s	remaining: 4.18s
346:	learn: 11.5713282	total: 2.21s	remaining: 4.17s
347:	learn: 11.5706251	total: 2.22s	remaining: 4.16s
348:	learn: 11.5697360	total: 2.23s	remaining: 4.15s
349:	learn: 11.5689485	total: 2.23s	remaining: 4.14s
350:	learn: 11.5680493	total: 2.24s	remaining: 4.14s
351:	learn: 11.5669729	total: 2.24s	remaining: 4.13s
352:	learn: 11.5661438	total: 2.25s	remaining: 4.

513:	learn: 11.4603794	total: 3.22s	remaining: 3.04s
514:	learn: 11.4596300	total: 3.23s	remaining: 3.04s
515:	learn: 11.4588639	total: 3.23s	remaining: 3.03s
516:	learn: 11.4580297	total: 3.23s	remaining: 3.02s
517:	learn: 11.4575193	total: 3.24s	remaining: 3.01s
518:	learn: 11.4569866	total: 3.25s	remaining: 3.01s
519:	learn: 11.4564588	total: 3.25s	remaining: 3s
520:	learn: 11.4554548	total: 3.26s	remaining: 3s
521:	learn: 11.4550808	total: 3.26s	remaining: 2.99s
522:	learn: 11.4546118	total: 3.27s	remaining: 2.98s
523:	learn: 11.4540470	total: 3.28s	remaining: 2.98s
524:	learn: 11.4539712	total: 3.28s	remaining: 2.97s
525:	learn: 11.4528287	total: 3.29s	remaining: 2.96s
526:	learn: 11.4525055	total: 3.29s	remaining: 2.96s
527:	learn: 11.4518653	total: 3.3s	remaining: 2.95s
528:	learn: 11.4511716	total: 3.31s	remaining: 2.94s
529:	learn: 11.4506034	total: 3.31s	remaining: 2.94s
530:	learn: 11.4504061	total: 3.32s	remaining: 2.93s
531:	learn: 11.4496502	total: 3.32s	remaining: 2.92s


696:	learn: 11.3696120	total: 4.31s	remaining: 1.88s
697:	learn: 11.3690983	total: 4.32s	remaining: 1.87s
698:	learn: 11.3687420	total: 4.33s	remaining: 1.86s
699:	learn: 11.3681386	total: 4.33s	remaining: 1.86s
700:	learn: 11.3677499	total: 4.34s	remaining: 1.85s
701:	learn: 11.3676927	total: 4.34s	remaining: 1.84s
702:	learn: 11.3675190	total: 4.35s	remaining: 1.84s
703:	learn: 11.3669919	total: 4.36s	remaining: 1.83s
704:	learn: 11.3668189	total: 4.36s	remaining: 1.82s
705:	learn: 11.3662460	total: 4.37s	remaining: 1.82s
706:	learn: 11.3657798	total: 4.37s	remaining: 1.81s
707:	learn: 11.3655958	total: 4.38s	remaining: 1.8s
708:	learn: 11.3650498	total: 4.38s	remaining: 1.8s
709:	learn: 11.3647104	total: 4.39s	remaining: 1.79s
710:	learn: 11.3646515	total: 4.4s	remaining: 1.79s
711:	learn: 11.3642353	total: 4.4s	remaining: 1.78s
712:	learn: 11.3636756	total: 4.41s	remaining: 1.77s
713:	learn: 11.3631400	total: 4.41s	remaining: 1.77s
714:	learn: 11.3627235	total: 4.42s	remaining: 1.7

852:	learn: 11.3100533	total: 5.21s	remaining: 898ms
853:	learn: 11.3095362	total: 5.22s	remaining: 893ms
854:	learn: 11.3093358	total: 5.23s	remaining: 887ms
855:	learn: 11.3091278	total: 5.23s	remaining: 881ms
856:	learn: 11.3090634	total: 5.24s	remaining: 874ms
857:	learn: 11.3089502	total: 5.25s	remaining: 868ms
858:	learn: 11.3089300	total: 5.25s	remaining: 862ms
859:	learn: 11.3082297	total: 5.26s	remaining: 856ms
860:	learn: 11.3081818	total: 5.26s	remaining: 850ms
861:	learn: 11.3079986	total: 5.27s	remaining: 844ms
862:	learn: 11.3075838	total: 5.28s	remaining: 838ms
863:	learn: 11.3071910	total: 5.29s	remaining: 833ms
864:	learn: 11.3068351	total: 5.3s	remaining: 827ms
865:	learn: 11.3065968	total: 5.3s	remaining: 821ms
866:	learn: 11.3063624	total: 5.31s	remaining: 815ms
867:	learn: 11.3059772	total: 5.32s	remaining: 809ms
868:	learn: 11.3053000	total: 5.33s	remaining: 803ms
869:	learn: 11.3047108	total: 5.33s	remaining: 797ms
870:	learn: 11.3045935	total: 5.34s	remaining: 7

In [21]:
def convert(pred,t_ori,outcome,flag):
    if (flag == 'treat1' and t_ori == '1') or (flag == 'treat2' and t_ori == '2') or (flag == 'treat0' and t_ori == '0'):
        return outcome
    else:
        return pred

train['t1_final'] = train.apply(lambda x:convert(x['t1_pred'],x['t_ori'],x['outcome'],'treat1'),axis=1) 
train['t2_final'] = train.apply(lambda x:convert(x['t2_pred'],x['t_ori'],x['outcome'],'treat2'),axis=1) 
train['t0_final'] = train.apply(lambda x:convert(x['t0_pred'],x['t_ori'],x['outcome'],'treat0'),axis=1) 

In [22]:
train['ce_1'] = train['t1_final'] - train['t0_final']
train['ce_2'] = train['t2_final'] - train['t0_final']

In [23]:
test['ce_1'] = test['t1_pred'] - test['t0_pred']
test['ce_2'] = test['t2_pred'] - test['t0_pred']

In [24]:
train[['ce_1','ce_2']].append(test[['ce_1','ce_2']]).to_csv('result.csv',index=False)

C:\temp\ipykernel_17196\1022380385.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train[['ce_1','ce_2']].append(test[['ce_1','ce_2']]).to_csv('result.csv',index=False)


In [26]:
train

,V_2,V_9,V_15,V_28,V_8,V_31,V_36,V_12,treatment,t1_pred,t2_pred,t0_pred,t_ori,outcome,t1_final,t2_final,t0_final,ce_1,ce_2
0,0.0,-5.175898,1,-0.385026,0,-1.399182,1.246007,3.0,0,2.421483,3.536832,2.993614,2,4.745129,2.421483,3.536832,2.993614,-0.572131,0.543218
1,0.0,0.193801,1,-0.475041,0,1.190330,0.958077,3.0,0,-4.234238,8.737212,-4.633088,0,7.310963,-4.234238,8.737212,-4.633088,0.398849,13.370300
2,1.0,0.193801,0,1.266557,0,-0.104426,-0.289619,3.0,0,-40.387450,-36.954384,-43.795149,2,-52.156193,-40.387450,-36.954384,-43.795149,3.407700,6.840765
3,0.0,0.193801,0,0.190288,0,-1.399182,-0.481573,3.0,0,-28.447286,-17.638609,-28.991904,0,-17.011566,-28.447286,-17.638609,-28.991904,0.544618,11.353295
4,0.0,0.193801,0,-0.882066,0,-0.536011,1.725891,6.0,0,-24.554107,-6.352762,-24.971999,2,-15.229788,-24.554107,-6.352762,-24.971999,0.417891,18.619237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36183,0.0,0.193801,0,-0.130635,0,0.758745,-0.673526,3.0,0,-14.206466,-2.137515,-14.939073,2,20.512318,-14.206466,-2.137515,-14.939073,0.732607,12.801558
36184,0.0,0.193801,0,2.119744,0,-0.536011,0.190264,6.0,0,-6.875171,-2.992956,-9.530718,0,-3.178066,-6.875171,-2.992956,-9.530718,2.655547,6.537762
36185,0.0,0.193801,0,-0.756828,0,-1.399182,0.574171,2.0,0,-34.274563,-16.656124,-34.183361,0,-43.733190,-34.274563,-16.656124,-34.183361,-0.091202,17.527237
36186,0.0,0.193801,0,-0.439818,0,0.758745,-0.097666,5.0,0,-7.410610,6.839882,-7.663844,2,12.905083,-7.410610,6.839882,-7.663844,0.253234,14.503726
